In [1]:

import pandas as pd
import numpy as np
from sklearn import metrics, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")


data_path = 'data/'

#lgb25 = pd.read_csv('../lgCV_2505.csv.gz', compression='gzip')
lgb27 = pd.read_csv('../lgCV_2705B.csv.gz', compression='gzip')
lgb31 = pd.read_csv('../lgCV_3105.csv.gz', compression='gzip')
rnn = pd.read_csv('../rnnCV_2805.csv.gz', compression='gzip')
rnn27 = pd.read_csv('../rnnCV_2705A.csv.gz', compression='gzip')
mlp = pd.read_csv('../mlpCV_2505.csv.gz', compression='gzip')
truth = pd.read_csv('../../train.csv.zip', compression='zip', parse_dates = ["activation_date"])
y = truth['deal_probability'].values
truth.drop('deal_probability', 1)
test =  pd.read_csv('../../test.csv.zip', compression='zip', parse_dates = ["activation_date"])
test['deal_probability']=float('NAN') 
truth = pd.concat([truth,test[truth.columns]],axis=0)

In [2]:
#rnn27trn = pd.read_pickle('../../features/y_pred_trn.pkl')
#rnn27tst = pd.read_csv('../rnndhsub_2705A.csv.gz', compression='gzip')
#rnn27 = pd.concat([rnn27tst, rnn27trn.reset_index().rename(columns = {0:'deal_probability'})], axis = 0)
#rnn27.to_csv('../rnnCV_2705A.csv.gz', compression='gzip')

In [3]:
#lgb25.rename(columns={'deal_probability': 'lgb25_preds' }, inplace=True)
lgb31.rename(columns={'deal_probability': 'lgb31_preds' }, inplace=True)
lgb27.rename(columns={'deal_probability': 'lgb27_preds' }, inplace=True)
rnn27.rename(columns={'deal_probability': 'rnn27_preds' }, inplace=True)
mlp.rename(columns={'deal_probability': 'mlp_preds' }, inplace=True)
preds_df = lgb27.merge(rnn, on='item_id')\
                .merge(mlp, on='item_id')\
                .merge(lgb31, on='item_id')\
                .merge(rnn27, on='item_id')\
                .merge(truth, on='item_id',how='left')

In [4]:
preds_df['difference_l27_r']=preds_df['lgb27_preds'] - preds_df['rnn_preds']
preds_df['difference_l27_r1']=preds_df['lgb27_preds'] - preds_df['rnn27_preds']
preds_df['difference_l27_m']=preds_df['lgb27_preds'] - preds_df['mlp_preds']
preds_df['difference_r_m']=preds_df['rnn27_preds'] - preds_df['mlp_preds']
#preds_df['difference_l27_l25']=preds_df['lgb27_preds'] - preds_df['lgb25_preds']
preds_df['price'].fillna(-1,inplace=True)

preds_df['max'] = np.max(np.array([preds_df[col] for col in preds_df.columns if '_preds' in col]),axis=0)
preds_df['min'] = np.min(np.array([preds_df[col] for col in preds_df.columns if '_preds' in col]),axis=0)
preds_df['avg'] = np.mean(np.array([preds_df[col] for col in preds_df.columns if '_preds' in col]),axis=0)
preds_df['std'] = np.std(np.array([preds_df[col] for col in preds_df.columns if '_preds' in col]),axis=0)

In [5]:
preds_df.head()

,item_id,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,Unnamed: 0,rnn27_preds,user_id,region,city,...,image_top_1,deal_probability,difference_l27_r,difference_l27_r1,difference_l27_m,difference_r_m,max,min,avg,std
0,b912c3c6a6ad,0.091656,0.069891,0.091223,0.083219,0,0.068331,e00f8ff2eaf9,Свердловская область,Екатеринбург,...,1008.0,0.12789,0.021765,0.023325,0.000433,-0.022892,0.091656,0.068331,0.080864,0.010068
1,2dac0150717d,0.147801,0.075968,0.211844,0.157973,1,0.070638,39aeb48f0017,Самарская область,Самара,...,692.0,0.00000,0.071833,0.077163,-0.064043,-0.141206,0.211844,0.070638,0.132845,0.053293
2,ba83aefab5dc,0.186261,0.167167,0.236579,0.227048,2,0.161924,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,...,3032.0,0.43177,0.019094,0.024336,-0.050318,-0.074655,0.236579,0.161924,0.195796,0.030652
3,02996f1dd2ea,0.241699,0.260876,0.383699,0.231447,3,0.278804,bf5cccea572d,Татарстан,Набережные Челны,...,796.0,0.80323,-0.019177,-0.037105,-0.142000,-0.104895,0.383699,0.231447,0.279305,0.054667
4,7c90be56d2ab,0.407423,0.431520,0.420657,0.412563,4,0.350284,ef50846afc0b,Волгоградская область,Волгоград,...,2264.0,0.20797,-0.024096,0.057139,-0.013234,-0.070373,0.431520,0.350284,0.404489,0.028298


### Train scores

In [6]:
idx = preds_df['deal_probability']==preds_df['deal_probability']
print(idx.value_counts())
for col in [c for c in preds_df.columns if '_preds' in c]:
    print('RMSE %s: '%(col), np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, preds_df[col][idx].values)))

True     1503424
False     508438
Name: deal_probability, dtype: int64
RMSE lgb27_preds:  0.21681527582821059
RMSE rnn_preds:  0.21771476573780568
RMSE mlp_preds:  0.2187544351068867
RMSE lgb31_preds:  0.21681135578306754
RMSE rnn27_preds:  0.21698040871120608


In [7]:
foldls = [["2017-03-15", "2017-03-16", "2017-03-17"], \
       ["2017-03-18", "2017-03-19", "2017-03-20"], \
       ["2017-03-21", "2017-03-22", "2017-03-23"], \
       ["2017-03-24", "2017-03-25", "2017-03-26"], \
        ["2017-03-27", "2017-03-28", "2017-03-29", \
            "2017-03-30", "2017-03-31", "2017-04-01", \
            "2017-04-02", "2017-04-03","2017-04-07"]]
foldls = [[pd.to_datetime(d) for d in f] for f in foldls]
preds_df['fold'] = -1
for t, fold in enumerate(foldls):
    preds_df['fold'][preds_df.activation_date.isin(fold)] = t

In [8]:
scores = []
for col in [c for c in preds_df.columns if '_preds' in c]:
    lstmp = [col]
    for i in range(5):
        idx = preds_df['fold']==i
        lstmp.append(np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, \
                                                        preds_df[col][idx].values)))
    scores.append(lstmp)
pd.DataFrame(scores, columns = ['Model']+['Fold%s'%(i) for i in range(5)])

,Model,Fold0,Fold1,Fold2,Fold3,Fold4
0,lgb27_preds,0.216848,0.217096,0.217923,0.217597,0.213692
1,rnn_preds,0.218756,0.218966,0.215764,0.219554,0.214747
2,mlp_preds,0.219660,0.220088,0.216672,0.220347,0.216385
3,lgb31_preds,0.216537,0.216643,0.219230,0.217169,0.213430
4,rnn27_preds,0.218097,0.218115,0.214982,0.218651,0.214379


### Correlations in test and train

In [9]:
# Test correlation
preds_df[~preds_df['deal_probability'].isnull()][[c for c in preds_df.columns if '_preds' in c]].corr()

,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,rnn27_preds
lgb27_preds,1.000000,0.915952,0.919171,0.990750,0.916804
rnn_preds,0.915952,1.000000,0.929710,0.910602,0.985360
mlp_preds,0.919171,0.929710,1.000000,0.914307,0.925010
lgb31_preds,0.990750,0.910602,0.914307,1.000000,0.914204
rnn27_preds,0.916804,0.985360,0.925010,0.914204,1.000000


In [10]:
# Train correlation
preds_df[preds_df['deal_probability'].isnull()][[c for c in preds_df.columns if '_preds' in c]].corr()

,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,rnn27_preds
lgb27_preds,1.000000,0.930288,0.924283,0.991439,0.930692
rnn_preds,0.930288,1.000000,0.939951,0.927835,0.988198
mlp_preds,0.924283,0.939951,1.000000,0.922413,0.935069
lgb31_preds,0.991439,0.927835,0.922413,1.000000,0.931468
rnn27_preds,0.930692,0.988198,0.935069,0.931468,1.000000


In [11]:
cols = [c for c in preds_df.columns if '_preds' in c]
cols += [c for c in preds_df.columns if 'difference' in c]
cols += ['price', 'max', 'min', 'avg', 'std']
categories = ['region', 'param_1', 'parent_category_name', 'category_name', 'param_2' ]
cols += categories
for col in categories:
    preds_df[col] = LabelEncoder().fit_transform(preds_df[col].fillna("0"))
train_df = preds_df[~preds_df['deal_probability'].isnull()]
test_df = preds_df[preds_df['deal_probability'].isnull()]

In [12]:
n_estimators = 4000
train_X, valid_X, train_y, valid_y = train_test_split(train_df[cols], y, train_size=.8, random_state=12345)
eval_set = [(train_X,train_y),(valid_X,valid_y)]

In [13]:
len(train_X)

1202739

In [14]:
%%time
clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.5,
                    num_leaves=32, 
                    learning_rate=.01, device='gpu')
clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=categories)

Training until validation scores don't improve for 80 rounds.
[100]	training's rmse: 0.220897	valid_1's rmse: 0.221222
[200]	training's rmse: 0.214655	valid_1's rmse: 0.215217
[300]	training's rmse: 0.213541	valid_1's rmse: 0.214305
[400]	training's rmse: 0.213202	valid_1's rmse: 0.214139
[500]	training's rmse: 0.213007	valid_1's rmse: 0.214089
[600]	training's rmse: 0.212836	valid_1's rmse: 0.214056
[700]	training's rmse: 0.212678	valid_1's rmse: 0.214029
[800]	training's rmse: 0.212539	valid_1's rmse: 0.214012
[900]	training's rmse: 0.212415	valid_1's rmse: 0.214002
[1000]	training's rmse: 0.212298	valid_1's rmse: 0.213991
[1100]	training's rmse: 0.212177	valid_1's rmse: 0.213984
[1200]	training's rmse: 0.212071	valid_1's rmse: 0.213978
[1300]	training's rmse: 0.211961	valid_1's rmse: 0.213971
[1400]	training's rmse: 0.211855	valid_1's rmse: 0.213967
[1500]	training's rmse: 0.211752	valid_1's rmse: 0.213962
[1600]	training's rmse: 0.211651	valid_1's rmse: 0.213959
[1700]	training's r

In [15]:
sorted(zip(clf.feature_importances_, train_X.columns ),key=lambda x: -x[0])

[(10963, 'param_1'),
 (6906, 'region'),
 (6756, 'param_2'),
 (3338, 'category_name'),
 (3017, 'lgb31_preds'),
 (2894, 'difference_l27_m'),
 (2885, 'rnn27_preds'),
 (2839, 'difference_l27_r1'),
 (2805, 'avg'),
 (2764, 'difference_r_m'),
 (2728, 'mlp_preds'),
 (2728, 'difference_l27_r'),
 (2403, 'min'),
 (2378, 'price'),
 (2362, 'std'),
 (2241, 'lgb27_preds'),
 (2224, 'rnn_preds'),
 (1739, 'max'),
 (541, 'parent_category_name')]

In [16]:
n_estimators = 2050
train_X = train_df[cols]
train_y = y
eval_set = [(train_X,train_y)]
len(train_X)

1503424

In [17]:
%%time
clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.5,
                    num_leaves=32, 
                    learning_rate=.01)#, device='gpu')

clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=categories)

Training until validation scores don't improve for 80 rounds.
[100]	training's rmse: 0.220967
[200]	training's rmse: 0.214767
[300]	training's rmse: 0.213683
[400]	training's rmse: 0.213367
[500]	training's rmse: 0.213193
[600]	training's rmse: 0.213046
[700]	training's rmse: 0.212911
[800]	training's rmse: 0.212792
[900]	training's rmse: 0.212687
[1000]	training's rmse: 0.212582
[1100]	training's rmse: 0.212483
[1200]	training's rmse: 0.21239
[1300]	training's rmse: 0.212298
[1400]	training's rmse: 0.21221
[1500]	training's rmse: 0.212121
[1600]	training's rmse: 0.212035
[1700]	training's rmse: 0.211949
[1800]	training's rmse: 0.211867
[1900]	training's rmse: 0.211787
[2000]	training's rmse: 0.211703
Did not meet early stopping. Best iteration is:
[2050]	training's rmse: 0.211662
CPU times: user 18min 42s, sys: 2.58 s, total: 18min 44s
Wall time: 2min 34s


In [18]:
test_df['deal_probability'] = clf.predict(test_df[cols])
test_df['deal_probability'] = np.clip(test_df['deal_probability'], .0001, .9999)
test_df[['item_id', 'deal_probability']].to_csv('../lgbbsub_0106EL2.csv.gz', compression='gzip', index=False, header=True)

In [19]:
test_df[['item_id', 'deal_probability']].head()

,item_id,deal_probability
1503424,6544e41a8817,0.386069
1503425,65b9484d670f,0.146470
1503426,8bab230b2ecd,0.126491
1503427,8e348601fefc,0.057894
1503428,8bd2fe400b89,0.228321
